### Setting up

In [ ]:
import numpy as np
import pandas as pd
import PIL
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import pathlib
import cv2

from mpl_toolkits.axes_grid1 import ImageGrid

In [ ]:
print(tf.__version__)

### Data

In [ ]:
path_train = '../input/plant-seedlings-classification/train'
data_dir = pathlib.Path(path_train)
folder = list(data_dir.glob('*'))
images = list(data_dir.glob('*/*.png')) #list of all images (full path)
print('Folder Structure:')
for f in folder:
    print(f)
print('\nNumber of images: ', len(images))

### Data Explore

In [ ]:
fig = plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.title(str(images[i]).split('/')[-1], fontsize=10) #get the file name and disply as title
    plt.imshow(PIL.Image.open(images[i]))
    ax = plt.axis("off")

### Preparing Data for Modeling

In [ ]:
image_size = 256
batch_size = 32

In [ ]:
idg = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2
)

In [ ]:
train_gen = idg.flow_from_directory(path_train,
                                    target_size=(image_size, image_size),
                                    subset='training',
                                    class_mode='categorical',
                                    batch_size=batch_size,
                                    shuffle=True,
                                    seed=1
                                    )

In [ ]:
val_gen = idg.flow_from_directory(path_train,
                                  target_size=(image_size, image_size),                                                   
                                  subset='validation',
                                  class_mode='categorical',
                                  batch_size=batch_size,
                                  shuffle=True,
                                  seed=1
                                  )

### Classify Train Data

In [ ]:
classes = train_gen.class_indices
print(classes)
class_names = []
for c in classes:
    class_names.append(c)
print('The name of the classes are: ', class_names)

### Explore Classification

In [ ]:
unique, counts = np.unique(train_gen.classes, return_counts=True)
dict1 = dict(zip(train_gen.class_indices, counts))

keys = dict1.keys()
values = dict1.values()

plt.xticks(rotation='vertical')
bar = plt.bar(keys, values)

In [ ]:
x,y = next(train_gen)

In [ ]:
fig = plt.figure(None, (10,10),frameon=False)
grid = ImageGrid(fig, 111, 
                 nrows_ncols=(2, 4),  
                 axes_pad=0.2, 
                 share_all=True,
                 )
for i in range(2*4):
    ax = grid[i]
    ax.imshow(x[i],cmap='Greys_r')
    ax.axis('off')        

### Modeling

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(image_size,image_size,3,))) # Input layer
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu')) # 2D Convolution layer
model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2))) # Max Pool layer 
model.add(tf.keras.layers.BatchNormalization()) # Normalization layer
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), strides = (1,1), activation='relu')) # 2D Convolution layer
model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2))) # Max Pool layer 
model.add(tf.keras.layers.BatchNormalization()) # Normalization layer
model.add(tf.keras.layers.Conv2D(128, kernel_size=(3,3), strides = (1,1), activation='relu')) # 2D Convolution layer
model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2))) # Max Pool layer 
model.add(tf.keras.layers.BatchNormalization()) # Normalization layer
model.add(tf.keras.layers.Conv2D(128, kernel_size=(3,3), strides = (1,1), activation='relu')) # 2D Convolution layer
model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2))) # Max Pool layer 
model.add(tf.keras.layers.GlobalMaxPool2D()) # Global Max Pool layer
model.add(tf.keras.layers.Flatten()) # Dense Layers after flattening the data
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2)) # Dropout
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.BatchNormalization()) # Normalization layer
model.add(tf.keras.layers.Dense(12, activation='softmax')) # Add Output Layer

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('plant_classifier.h5', #where to save the model
                                                save_best_only=True, 
                                                monitor='val_accuracy', 
                                                mode='max', 
                                                verbose = 1)

### Training

In [ ]:
activity = model.fit(train_gen,
          epochs=20, # Increase number of epochs if you have sufficient hardware
          steps_per_epoch= 3803//batch_size,  # Number of train images // batch_size
          validation_data=val_gen,
          validation_steps = 947//batch_size, # Number of val images // batch_size
          callbacks = [checkpoint],
          verbose = 1
)

In [ ]:
plt.plot(activity.history['accuracy'], label='accuracy')
plt.plot(activity.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.xticks(list(range(1,21)))
plt.ylim([0, 1])
plt.legend(loc='lower right')

### Prediction

In [ ]:
maize = cv2.imread(path_train+'/Maize/6e9ff31e7.png')
ax = plt.imshow(maize)

In [ ]:
maize = cv2.resize(maize, (256,256))
maize_batch = np.expand_dims(maize, axis=0)
conv_maize = model.predict(maize_batch)
conv_maize.shape

In [ ]:
score = tf.nn.softmax(conv_maize[0])

In [ ]:
simple_model = tf.keras.models.Sequential()
simple_model.add(tf.keras.layers.Conv2D(1,3,3,input_shape=maize.shape)) # 3x3 kernel

maize_batch = np.expand_dims(maize, axis=0)
conv_maize2 = simple_model.predict(maize_batch)
conv_maize2 = np.squeeze(conv_maize2, axis=0)
    
print(conv_maize2.shape)
conv_maize2 = conv_maize2.reshape(conv_maize2.shape[:2])
print(conv_maize2.shape)

ax = plt.imshow(conv_maize2)
ax = plt.title("This is a image of {} ({:.2f}% confidence).".format(class_names[np.argmax(score)], 100 * np.max(score)), fontsize=12)